# Linear SVM Test
##### https://woolulu.tistory.com/category/Tensorflow/Support%20Vector%20Machine
##### https://medium.com/cs-note/tensorflow-ch4-support-vector-machines-c9ad18878c76

In [1]:
import tensorflow as tf
tf.__version__

'1.14.0'

In [ ]:
# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split
# import numpy as np

# iris = load_iris()
# x_vals = iris.data[:, [0,3]]
# y_vals = np.array([1 if y==0 else -1 for y in iris.target])
# x_train, x_test, y_train, y_test = train_test_split(x_vals, y_vals, random_state=0, stratify=y_vals)

import tensorflow as tf
from tensorflow.python.framework import ops

import numpy as np

import fl_data

ops.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

DATA_NAME = 'cifar10' # mnist-o / mnist-f / cifar-10
train_data, test_data = tf.keras.datasets.cifar10.load_data()

train_dataBy1Nid = fl_data.preprocess('svm', DATA_NAME, train_data, True)
test_dataBy1Nid = fl_data.preprocess('svm', DATA_NAME, test_data, True)
print(train_dataBy1Nid[0]['x'].shape, test_dataBy1Nid[0]['x'].shape)

sess = tf.Session(config=config)

# placeholder
x = tf.placeholder(shape=[None, 3072], dtype=tf.float32)
y = tf.placeholder(shape=[None], dtype=tf.int32)

# create Variable
A = tf.Variable(tf.zeros(shape=[3072, 1], dtype=tf.float32))
b = tf.Variable(tf.zeros(shape=[1], dtype=tf.float32))

# formula (y = ax - b)
formula = tf.matmul(x, A) - b

# declare loss function
# Loss = summation(max(0, 1 - pred*actual)) + alpha * L2_norm(A)^2
y_reshaped = tf.reshape(y, [-1, 1])
loss = tf.reduce_mean(tf.maximum(0.0, 1.0 - formula * tf.cast(y_reshaped, tf.float32)))

y_hat = tf.squeeze(tf.cast(tf.sign(formula), tf.int32))
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_hat, y), tf.float32))
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# initialize variable
sess.run(tf.global_variables_initializer())

x_train = train_dataBy1Nid[0]['x'][0:2000]
y_train = train_dataBy1Nid[0]['y'][0:2000]
x_test = test_dataBy1Nid[0]['x'][0:10000]
y_test = test_dataBy1Nid[0]['y'][0:10000]

# x = x_train
# y = y_train.reshape(-1, 1)
for t in np.arange(100000):
    sess.run(train_op, feed_dict={x: x_train, y:y_train})
    if t % 50 == 0:
#        vs = sess.run(tf.concat([ tf.reshape(layer, [-1]) for layer in tf.get_collection(tf.GraphKeys.VARIABLES) ], axis=0))
#        numVars = vs.shape ; print(numVars)
        (loss_, accuracy_) = sess.run((loss, accuracy), feed_dict={x: x_test, y:y_test})
        print('{},{},{}'.format(t, loss_, accuracy_))
    if accuracy_ >= 0.9:
        break

W1006 19:24:36.482066 140145767819008 deprecation.py:323] From /home/dmlab/prosopher/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(50000, 3072) (10000, 3072)
0,0.8555577993392944,0.6000000238418579
50,0.7705511450767517,0.7020000219345093
100,0.615459144115448,0.7545999884605408
150,0.6015509963035583,0.7613999843597412
200,0.5836949944496155,0.7674999833106995
250,0.5728937983512878,0.7700999975204468
300,0.5584739446640015,0.7771000266075134
350,0.5655651092529297,0.7753000259399414
400,0.5490298271179199,0.781000018119812
450,0.5702677965164185,0.7753999829292297
500,0.5617369413375854,0.7782999873161316
550,0.5505491495132446,0.7821999788284302
600,0.5583293437957764,0.7814000248908997
650,0.5497826933860779,0.7835000157356262
700,0.5454190969467163,0.7850000262260437
750,0.5351250767707825,0.7883999943733215
800,0.5536011457443237,0.784500002861023
850,0.5317941308021545,0.7890999913215637
900,0.5577540397644043,0.7839999794960022
950,0.5423465371131897,0.7870000004768372
1000,0.5167508721351624,0.794700026512146
1050,0.5468797087669373,0.7868000268936157
1100,0.558967113494873,0.7838000059127808
1150,0.5611

# Gaussian SVM Test
#### https://github.com/mjc13813759744/machine-Learning/blob/master/non_linear%20SVM(RBF)%20by%20tensorflow

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops

ops.reset_default_graph()

sess = tf.Session()

import fl_data

BATCH_SIZE = 512
DATA_NAME = 'cifar10' # mnist-o / mnist-f / cifar-10
train_data, test_data = tf.keras.datasets.cifar10.load_data()

train_dataBy1Nid = fl_data.preprocess('svm', DATA_NAME, train_data, True)
test_dataBy1Nid = fl_data.preprocess('svm', DATA_NAME, test_data, True)
print(train_dataBy1Nid[0]['x'].shape, test_dataBy1Nid[0]['x'].shape)

x_train = train_dataBy1Nid[0]['x']
y_train = train_dataBy1Nid[0]['y']
x_test = test_dataBy1Nid[0]['x']
y_test = test_dataBy1Nid[0]['y']

# (x_vals_, y_vals_) = datasets.make_circles(n_samples=350, factor=.5, noise=.1)
# y_vals_ = np.array([1 if y==1 else -1 for y in y_vals_])
# print(x_vals_.shape, y_vals.shape)

x = tf.placeholder(shape=[None, 3072], dtype=tf.float32)
y = tf.placeholder(shape=[None], dtype=tf.int32)
y_reshaped = tf.reshape(y, [-1, 1])

W = tf.Variable(tf.random_normal(shape=[1, 3072]))
b = tf.Variable(tf.random_normal(shape=[1, 1]))

#compute Gaussian Kernel 
gamma = tf.constant(-10.0)

#dist = tf.reduce_sum(tf.square(x), 1)
#dist = tf.reshape(dist, [-1, 1])
#sq_dists = tf.add(tf.subtract(dist, tf.multiply(2., tf.matmul(x, tf.transpose(x)))), tf.transpose(dist))
dist = tf.reduce_sum(tf.square(tf.transpose(x)), 1)
dist = tf.reshape(dist, [-1, 1])
sq_dists = tf.add(tf.subtract(dist, tf.multiply(2., tf.matmul(tf.transpose(x), x))), tf.transpose(dist))
my_kernel = tf.exp(tf.multiply(gamma, sq_dists))
output = tf.add(tf.matmul(W, my_kernel), b)
print(tf.shape(output))

y_float = tf.cast(y, tf.float32)
loss = tf.reduce_mean(tf.maximum(0.,tf.subtract(1., tf.multiply(output, y_float)))) + tf.matmul(W, tf.transpose(W))

# x_trainfloat = x #tf.to_float(x_train)

# pred_dist = tf.reduce_sum(tf.square(x_trainfloat), 1)
# pred_dist = tf.reshape(pred_dist, [-1, 1])
# pred_sq_dists = tf.add(tf.subtract(pred_dist, tf.multiply(2., tf.matmul(x_trainfloat, tf.transpose(x_trainfloat)))), tf.transpose(pred_dist))
# pred_kernel = tf.exp(tf.multiply(gamma, pred_sq_dists))

# predict_train = tf.add(tf.matmul(W, my_kernel), b)
# prediction = tf.sign(predict_train)
y_hat = tf.squeeze(tf.cast(tf.sign(output), tf.int32))
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_hat, y), tf.float32))

train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

sess.run(tf.global_variables_initializer())

def next_batch(batchSize, x, y):
    idx = np.arange(0 , len(x))
    np.random.shuffle(idx)
    x_ = [x[i] for i in idx[:batchSize]]
    y_ = [y[i] for i in idx[:batchSize]]
    return x_, y_

# Training loop
for epoch in range(10000):
    trainBatch = next_batch(BATCH_SIZE, x_train, y_train)
    sess.run(train_op, feed_dict={x:trainBatch[0], y:trainBatch[1]})
    
#     if epoch % 1000 == 0:
#         (loss_, acc_) = sess.run((loss, accuracy), feed_dict={x:x_train, y:y_train})
#         print(epoch, loss_, acc_)
        
    if epoch % 100 == 0:
        numTestIters = int(10000/BATCH_SIZE)
        losses_ = [] ; accs_ = [] ; idxBegin = 0 ; idxEnd = 0
        for i in range(numTestIters):
            idxEnd += BATCH_SIZE
            sampleBatch_x = x_test[idxBegin:idxEnd]
            sampleBatch_y = y_test[idxBegin:idxEnd]
            (loss_, acc_) = sess.run((loss, accuracy), feed_dict={x: sampleBatch_x, y: sampleBatch_y})
            losses_.append(loss_)
            accs_.append(acc_)
            idxBegin = idxEnd
        (loss_, acc_) = np.mean(losses_), np.mean(accs_)
        #vs = sess.run(tf.concat([ tf.reshape(layer, [-1]) for layer in tf.get_collection(tf.GraphKeys.VARIABLES) ], axis=0))
        #numVars = vs.shape ; print(numVars)
        print(epoch, loss_, acc_)

ImportError: No module named 'sklearn'

# 9-layer CNN Test
##### https://www.tensorflow.org/tutorials/images/deep_cnn

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

import fl_data

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

import numpy as np

def preprocess(data):
    x = np.array([ x / 255.0 for x in data[0] ], dtype=np.float32)
    dataY = data[1].flatten() # cifar10 의 경우 flatten 필요
    y = np.array(dataY, dtype=np.int32)
    return np.array([ { 'x': x, 'y': y } ])

def next_batch(batchSize, x, y):
    idx = np.arange(0 , len(x))
    np.random.shuffle(idx)
    x_ = [x[i] for i in idx[:batchSize]]
    y_ = [y[i] for i in idx[:batchSize]]
    return x_, y_

def build_CNN_classifier(x):
    x = tf.expand_dims(x, axis=-1)
    
    # 첫번째 convolutional layer - 하나의 grayscale 이미지를 64개의 특징들(feature)으로 맵핑(maping)합니다.
    W_conv1 = tf.get_variable('W_conv1', dtype=tf.float32, initializer=tf.truncated_normal(shape=[5, 5, 1, 64], stddev=5e-2))
    b_conv1 = tf.get_variable('b_conv1', dtype=tf.float32, initializer=tf.constant(0.1, shape=[64]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

    # 첫번째 Pooling layer
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    norm1 = tf.nn.lrn(h_pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

    # 두번째 convolutional layer - 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)합니다.
    W_conv2 = tf.get_variable('W_conv2', dtype=tf.float32, initializer=tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
    b_conv2 = tf.get_variable('b_conv2', dtype=tf.float32, initializer=tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(norm1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
    norm2 = tf.nn.lrn(h_conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')

    # 두번째 pooling layer.
    h_pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    #   h_pool2_flatten = tf.reshape(h_pool2, [-1, 2048])
    h_pool2_flatten = tf.keras.layers.Flatten()(h_pool2)

    #   # 세번째 convolutional layer
    #   W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
    #   b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
    #   h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

    #   # 네번째 convolutional layer
    #   W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
    #   b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
    #   h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

    #   # 다섯번째 convolutional layer
    #   W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
    #   b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
    #   h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)
    #   h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])


    # Fully Connected Layer 1 - 2번의 downsampling 이후에, 우리의 32x32 이미지는 8x8x128 특징맵(feature map)이 됩니다.
    # 이를 384개의 특징들로 맵핑(maping)합니다.

    W_fc1 = tf.get_variable('W_fc1', dtype=tf.float32, initializer=tf.truncated_normal(shape=[3136, 512], stddev=5e-2))
    b_fc1 = tf.get_variable('b_fc1', dtype=tf.float32, initializer=tf.constant(0.1, shape=[512]))
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flatten, W_fc1) + b_fc1)

    #   W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 192], stddev=5e-2))
    #   b_fc2 = tf.Variable(tf.constant(0.1, shape=[192]))
    #   h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)  

    # Fully Connected Layer 2 - 384개의 특징들(feature)을 10개의 클래스-airplane, automobile, bird...-로 맵핑(maping)합니다.
    W_fc3 = tf.get_variable('W_fc3', dtype=tf.float32, initializer=tf.truncated_normal(shape=[512, 10], stddev=5e-2))
    b_fc3 = tf.get_variable('b_fc3', dtype=tf.float32, initializer=tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc1,W_fc3) + b_fc3
    y_prob = tf.nn.softmax(logits)
    y_hat = tf.cast(tf.argmax(y_prob, 1), tf.int32)
    return y_hat, logits

trainData, testData = tf.keras.datasets.mnist.load_data()

trainData_by1Nid = fl_data.preprocess('cnn', 'mnist-o', trainData, False)
testData_by1Nid = fl_data.preprocess('cnn', 'mnist-o', testData, False)
x_train = trainData_by1Nid[0]['x']
y_train = trainData_by1Nid[0]['y']
x_test = testData_by1Nid[0]['x']
y_test = testData_by1Nid[0]['y']
print(x_train.shape)

x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y = tf.placeholder(shape=[None], dtype=tf.int32)

y_hat, logits = build_CNN_classifier(x)
y_onehot = tf.one_hot(y, 10)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_onehot, logits=logits))
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_hat, y), tf.float32))

lr = tf.placeholder(tf.float32, shape=[])
train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss)

BATCH_SIZE = 128

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    lr_ = 0.01
    for t in range(100000):
        trainBatch = next_batch(BATCH_SIZE, x_train, y_train)
        testBatch = next_batch(BATCH_SIZE, x_test, y_test)
        sess.run(train_op, feed_dict={lr: lr_, x: trainBatch[0], y: trainBatch[1]})
        
        if t % 390 == 0:
            lr_ *= 0.99
            losses_ = [] ; accs_ = []
            for _ in range(78):
                (sampleBatch_x, sampleBatch_y) = next_batch(BATCH_SIZE, x_test, y_test)
                (loss_, acc_) = sess.run((loss, accuracy), feed_dict={x: sampleBatch_x, y: sampleBatch_y})
                losses_.append(loss_)
                accs_.append(acc_)
            print("%d\t%.4f\t%.3f\t%.3f" % (t, lr_, np.mean(losses_), np.mean(accs_)))

# CNN with CIFAR10
#### http://solarisailab.com/archives/2325

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

sess = tf.Session()

import fl_data

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

import numpy as np

def preprocess(data):
    x = np.array([ x / 255.0 for x in data[0] ], dtype=np.float32)
    dataY = data[1].flatten() # cifar10 의 경우 flatten 필요
    y = np.array(dataY, dtype=np.int32)
    return np.array([ { 'x': x, 'y': y } ])

def next_batch(batchSize, x, y):
    idx = np.arange(0 , len(x))
    np.random.shuffle(idx)
    x_ = [x[i] for i in idx[:batchSize]]
    y_ = [y[i] for i in idx[:batchSize]]
    return x_, y_

# CNN 모델을 정의합니다. 
def build_CNN_classifier(x):
    # 첫번째 convolutional layer - 하나의 grayscale 이미지를 64개의 특징들(feature)으로 맵핑(maping)합니다.
#     W_conv1 = tf.get_variable('W_conv1', dtype=tf.float32, initializer=tf.truncated_normal(shape=[5, 5, 3, 64], stddev=5e-2))
#     b_conv1 = tf.get_variable('b_conv1', dtype=tf.float32, initializer=tf.constant(0.1, shape=[64]))
    W_conv1 = tf.get_variable('W_conv1', dtype=tf.float32, initializer=tf.truncated_normal(shape=[3, 3, 3, 64], stddev=5e-2))
    b_conv1 = tf.get_variable('b_conv1', dtype=tf.float32, initializer=tf.constant(0.1, shape=[64]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='VALID') + b_conv1)
    
    # 첫번째 Pooling layer
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # 두번째 convolutional layer - 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)합니다.
#     W_conv2 = tf.get_variable('W_conv2', dtype=tf.float32, initializer=tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
#     b_conv2 = tf.get_variable('b_conv2', dtype=tf.float32, initializer=tf.constant(0.1, shape=[64]))
    W_conv2 = tf.get_variable('W_conv2', dtype=tf.float32, initializer=tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
    b_conv2 = tf.get_variable('b_conv2', dtype=tf.float32, initializer=tf.constant(0.1, shape=[128]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='VALID') + b_conv2)
    
    # 두번째 pooling layer.
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 세번째 convolutional layer
#     W_conv3 = tf.get_variable('W_conv3', dtype=tf.float32, initializer=tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
#     b_conv3 = tf.get_variable('b_conv3', dtype=tf.float32, initializer=tf.constant(0.1, shape=[128]))
    W_conv3 = tf.get_variable('W_conv3', dtype=tf.float32, initializer=tf.truncated_normal(shape=[3, 3, 128, 256], stddev=5e-2))
    b_conv3 = tf.get_variable('b_conv3', dtype=tf.float32, initializer=tf.constant(0.1, shape=[256]))
    h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='VALID') + b_conv3)

    # 네번째 convolutional layer
#     W_conv4 = tf.get_variable('W_conv4', dtype=tf.float32, initializer=tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
#     b_conv4 = tf.get_variable('b_conv4', dtype=tf.float32, initializer=tf.constant(0.1, shape=[128]))
    W_conv4 = tf.get_variable('W_conv4', dtype=tf.float32, initializer=tf.truncated_normal(shape=[3, 3, 256, 256], stddev=5e-2))
    b_conv4 = tf.get_variable('b_conv4', dtype=tf.float32, initializer=tf.constant(0.1, shape=[256]))
    h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='VALID') + b_conv4)

    # 다섯번째 convolutional layer
#     W_conv5 = tf.get_variable('W_conv5', dtype=tf.float32, initializer=tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
#     b_conv5 = tf.get_variable('b_conv5', dtype=tf.float32, initializer=tf.constant(0.1, shape=[128]))
    W_conv5 = tf.get_variable('W_conv5', dtype=tf.float32, initializer=tf.truncated_normal(shape=[3, 3, 256, 256], stddev=5e-2))
    b_conv5 = tf.get_variable('b_conv5', dtype=tf.float32, initializer=tf.constant(0.1, shape=[256]))
    h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='VALID') + b_conv5)

    # Fully Connected Layer 1 - 2번의 downsampling 이후에, 우리의 32x32 이미지는 8x8x128 특징맵(feature map)이 됩니다.
    # 이를 384개의 특징들로 맵핑(maping)합니다.
#     W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
#     b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[1 * 1 * 256, 128], stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[128]))

    h_conv5_flat = tf.reshape(h_conv5, [-1, 1 * 1 * 256])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

    # Dropout - 모델의 복잡도를 컨트롤합니다. 특징들의 co-adaptation을 방지합니다.
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

    # Fully Connected Layer 2 - 384개의 특징들(feature)을 10개의 클래스-airplane, automobile, bird...-로 맵핑(maping)합니다.
#     W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2))
#     b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
    W_fc2 = tf.Variable(tf.truncated_normal(shape=[128, 10], stddev=5e-2))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
    y_prob = tf.nn.softmax(logits)
    y_hat = tf.cast(tf.argmax(y_prob, 1), tf.int32)
    return y_hat, logits

# 인풋 아웃풋 데이터, 드롭아웃 확률을 입력받기위한 플레이스홀더를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.int32, shape=[None])
keep_prob = tf.placeholder(tf.float32)

# scalar 형태의 레이블(0~9)을 One-hot Encoding 형태로 변환합니다.
y_onehot = tf.one_hot(y, 10)

# Convolutional Neural Networks(CNN) 그래프를 생성합니다.
y_hat, logits = build_CNN_classifier(x)
y_onehot = tf.one_hot(y, 10)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_onehot, logits=logits))
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_hat, y), tf.float32))

lr = tf.placeholder(name='lr', shape=[], dtype=tf.float32)
train_step = tf.train.GradientDescentOptimizer(lr).minimize(loss)

# 모든 변수들을 초기화한다. 
sess.run(tf.global_variables_initializer())

trainData, testData = tf.keras.datasets.cifar10.load_data()

trainData_by1Nid = fl_data.preprocess('cnn', 'cifar10', trainData, False)
testData_by1Nid = fl_data.preprocess('cnn', 'cifar10', testData, False)
x_train = trainData_by1Nid[0]['x']
y_train = trainData_by1Nid[0]['y']
x_test = testData_by1Nid[0]['x'][0:10000]
y_test = testData_by1Nid[0]['y'][0:10000]
print(x_train.shape)

BATCH_SIZE = 128
lr_ = 0.01

# 10000 Step만큼 최적화를 수행합니다.
for i in range(1000000):
    batch = next_batch(BATCH_SIZE, x_train, y_train)

    # 20% 확률의 Dropout을 이용해서 학습을 진행합니다.
    sess.run(train_step, feed_dict={lr: lr_, x: batch[0], y: batch[1], keep_prob: 0.5})

    # 100 Step마다 training 데이터셋에 대한 정확도와 loss를 출력합니다.
    if i % 390 == 0:
        numTestIters = int(10000/BATCH_SIZE)
        losses_ = [] ; accs_ = [] ; idxBegin = 0 ; idxEnd = 0
        for _ in range(numTestIters):
            idxEnd += BATCH_SIZE
            sampleBatch_x = x_test[idxBegin:idxEnd]
            sampleBatch_y = y_test[idxBegin:idxEnd]
            (loss_, acc_) = sess.run((loss, accuracy), feed_dict={lr: lr_, x: sampleBatch_x, y: sampleBatch_y, keep_prob: 1.0})
            losses_.append(loss_)
            accs_.append(acc_)
            idxBegin = idxEnd
        (loss_, acc_) = np.mean(losses_), np.mean(accs_)
        #vs = sess.run(tf.concat([ tf.reshape(layer, [-1]) for layer in tf.get_collection(tf.GraphKeys.VARIABLES) ], axis=0))
        #numVars = vs.shape ; print(numVars)
        print(int(i / 390), loss_, acc_)
        
        lr_ *= 0.99

# Resnet 32
##### https://dnddnjs.github.io/cifar10/2018/10/09/resnet/

# 2NN Multi Layer Perceptron
##### https://towardsdatascience.com/multi-layer-perceptron-using-tensorflow-9f3e218a4809

In [ ]:
# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split
# import numpy as np

# iris = load_iris()
# x_vals = iris.data[:, [0,3]]
# y_vals = np.array([1 if y==0 else -1 for y in iris.target])
# x_train, x_test, y_train, y_test = train_test_split(x_vals, y_vals, random_state=0, stratify=y_vals)

import tensorflow as tf
from tensorflow.python.framework import ops

import numpy as np

import fl_data

ops.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

DATA_NAME = 'mnist-o' # mnist-o / mnist-f / cifar-10

if DATA_NAME == 'mnist-o':
    train_data, test_data = tf.keras.datasets.mnist.load_data()
elif DATA_NAME == 'mnist-f':
    train_data, test_data = tf.keras.datasets.fashion_mnist.load_data()
elif DATA_NAME == 'cifar10':
    train_data, test_data = tf.keras.datasets.cifar10.load_data()
else:
    raise Exception(DATA_NAME)
train_dataBy1Nid = fl_data.preprocess('sr', DATA_NAME, train_data, True)
test_dataBy1Nid = fl_data.preprocess('sr', DATA_NAME, test_data, True)
print(train_dataBy1Nid[0]['x'].shape, test_dataBy1Nid[0]['x'].shape)

sess = tf.Session(config=config)

# Placeholders for the input data
x = tf.placeholder(name='x', shape=[None, 784], dtype=tf.float32)
y = tf.placeholder(name='y', shape=[None], dtype=tf.int32)
y_onehot = tf.one_hot(y, 10)

num_features = 784
num_layers_0 = 200
num_layers_1 = 200
#regularizer_rate = 0.1

#keep_prob = tf.constant(0.6)

## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
tf.random.set_random_seed(1234) # 모든 노드가 같은 Initial Random Seed 를 갖지 않으면 학습되지 않음 (FedAvg 논문 참조)
W_0 = tf.get_variable('W_0', dtype=tf.float32, initializer=tf.random_normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
b_0 = tf.get_variable('b_0', dtype=tf.float32, initializer=tf.random_normal([num_layers_0]))
W_1 = tf.get_variable('W_1', dtype=tf.float32, initializer=tf.random_normal([num_layers_0,num_layers_1], stddev=(1/tf.sqrt(float(num_layers_0)))))
b_1 = tf.get_variable('b_1', dtype=tf.float32, initializer=tf.random_normal([num_layers_1]))
W_2 = tf.get_variable('W_2', dtype=tf.float32, initializer=tf.random_normal([num_layers_1,10], stddev=(1/tf.sqrt(float(num_layers_1)))))
b_2 = tf.get_variable('b_2', dtype=tf.float32, initializer=tf.random_normal([10]))

## Initializing weigths and biases
hidden_output_0 = tf.nn.relu(tf.matmul(x,W_0)+b_0)
#hidden_output_0_0 = tf.nn.dropout(hidden_output_0, keep_prob)
hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0,W_1)+b_1)
#hidden_output_1_1 = tf.nn.dropout(hidden_output_1, keep_prob)
logits = tf.matmul(hidden_output_1, W_2) + b_2
y_prob = tf.nn.softmax(logits)
y_hat = tf.cast(tf.argmax(y_prob, 1), tf.int32)

## Defining the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_onehot))# \
        #+ regularizer_rate*(tf.reduce_sum(tf.square(b_0)) + tf.reduce_sum(tf.square(b_1)))
train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(y_hat, y), tf.float32))

# initialize variable
sess.run(tf.global_variables_initializer())

x_train = train_dataBy1Nid[0]['x'][0:10000]
y_train = train_dataBy1Nid[0]['y'][0:10000]
x_test = test_dataBy1Nid[0]['x'][0:10000]
y_test = test_dataBy1Nid[0]['y'][0:10000]

# x = x_train
# y = y_train.reshape(-1, 1)
for t in np.arange(100000):
    sess.run(train_op, feed_dict={x: x_train, y:y_train})
    loss_ = sess.run(loss, feed_dict={x: x_train, y:y_train})
    accuracy_ = sess.run(accuracy, feed_dict={x: x_test, y:y_test})
    if t % 100 == 0:
        print('{},{},{}'.format(t, loss_, accuracy_))
    if accuracy_ >= 0.99:
        break